In [51]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [52]:
from typing import Generator, List
import pickle

from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import spacy
from spacy.lang.en import English

In [53]:
nlp = spacy.load('en_core_web_trf')

In [54]:
with open(r"data/interim/review_classes.pkl", "rb") as input_file:
    review_classes = pickle.load(input_file)

In [55]:
review_classes["NEG"]

["Casa del Arco is a very conveniently located bed and breakfast place in Antigua Guatemala. It's just two minute walking distance from Parque Central and the Arch of Catalina. In fact given its central location, just about every attraction in the town is accessible by walk. The place is a cosy nook with few rooms. That said, you can almost have their little lounge to yourself when you want. The rooms are tastefully done with Guatemalan textile and clean. The bed was comfortable as well. The only issue I had was the lack of ventilation. The room window opened to the steps just besides the room that would compromise my privacy but as I had mentioned, there are few rooms and very less people in the property so it was fine. The highlight of the place is the breakfast cooked by the",
 'Good location on the NW of town, especially if youre going to Antiguena spanish school. 100q for good sized private twin room with shared bathroom - great price Nice ish garden with a swing and hamock Basic 

In [56]:
positive_reviews = review_classes['POS']
negative_reviews = review_classes['NEG']

In [57]:
negative_reviews

["Casa del Arco is a very conveniently located bed and breakfast place in Antigua Guatemala. It's just two minute walking distance from Parque Central and the Arch of Catalina. In fact given its central location, just about every attraction in the town is accessible by walk. The place is a cosy nook with few rooms. That said, you can almost have their little lounge to yourself when you want. The rooms are tastefully done with Guatemalan textile and clean. The bed was comfortable as well. The only issue I had was the lack of ventilation. The room window opened to the steps just besides the room that would compromise my privacy but as I had mentioned, there are few rooms and very less people in the property so it was fine. The highlight of the place is the breakfast cooked by the",
 'Good location on the NW of town, especially if youre going to Antiguena spanish school. 100q for good sized private twin room with shared bathroom - great price Nice ish garden with a swing and hamock Basic 

In [58]:
def sentences_to_words(sentences: List[str]) -> List[List[str]]:
    words = []
    for sentence in sentences:
        # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
        words.append(simple_preprocess(str(sentence), deacc=True))  # deacc=True elimina la puntuación
    return words

In [59]:
print(negative_reviews[1])
negative_words = sentences_to_words(negative_reviews)
positive_words = sentences_to_words(positive_reviews)
print(negative_words[1])

Good location on the NW of town, especially if youre going to Antiguena spanish school. 100q for good sized private twin room with shared bathroom - great price Nice ish garden with a swing and hamock Basic bathrooms - very tatty but has hot showers always (if you keep the water flow slow) Super basic, pretty dirty kitchenette Loads of cats and dogs  Its very cheap and basic. If thats what your after, its ok
['good', 'location', 'on', 'the', 'nw', 'of', 'town', 'especially', 'if', 'youre', 'going', 'to', 'antiguena', 'spanish', 'school', 'for', 'good', 'sized', 'private', 'twin', 'room', 'with', 'shared', 'bathroom', 'great', 'price', 'nice', 'ish', 'garden', 'with', 'swing', 'and', 'hamock', 'basic', 'bathrooms', 'very', 'tatty', 'but', 'has', 'hot', 'showers', 'always', 'if', 'you', 'keep', 'the', 'water', 'flow', 'slow', 'super', 'basic', 'pretty', 'dirty', 'kitchenette', 'loads', 'of', 'cats', 'and', 'dogs', 'its', 'very', 'cheap', 'and', 'basic', 'if', 'thats', 'what', 'your', 'af

In [60]:
def remove_stopwords(documents: List[List[str]]) -> List[List[str]]:
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords.words('english')]
            for doc in documents]

In [61]:
print(negative_words[1])
negative_words_wo_stopwords = remove_stopwords(negative_words)
print(negative_words_wo_stopwords[1])

['good', 'location', 'on', 'the', 'nw', 'of', 'town', 'especially', 'if', 'youre', 'going', 'to', 'antiguena', 'spanish', 'school', 'for', 'good', 'sized', 'private', 'twin', 'room', 'with', 'shared', 'bathroom', 'great', 'price', 'nice', 'ish', 'garden', 'with', 'swing', 'and', 'hamock', 'basic', 'bathrooms', 'very', 'tatty', 'but', 'has', 'hot', 'showers', 'always', 'if', 'you', 'keep', 'the', 'water', 'flow', 'slow', 'super', 'basic', 'pretty', 'dirty', 'kitchenette', 'loads', 'of', 'cats', 'and', 'dogs', 'its', 'very', 'cheap', 'and', 'basic', 'if', 'thats', 'what', 'your', 'after', 'its', 'ok']
['good', 'location', 'nw', 'town', 'especially', 'youre', 'going', 'antiguena', 'spanish', 'school', 'good', 'sized', 'private', 'twin', 'room', 'shared', 'bathroom', 'great', 'price', 'nice', 'ish', 'garden', 'swing', 'hamock', 'basic', 'bathrooms', 'tatty', 'hot', 'showers', 'always', 'keep', 'water', 'flow', 'slow', 'super', 'basic', 'pretty', 'dirty', 'kitchenette', 'loads', 'cats', 'do

In [62]:
positive_words_wo_stopwords = remove_stopwords(positive_words)

In [63]:
def learn_bigrams(documents: List[List[str]]) -> List[List[str]]:
    # We learn bigrams
    #  https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases
    bigram = Phrases(documents, min_count=10, threshold=10)

    # we reduce the bigram model to its minimal functionality
    bigram_mod = Phraser(bigram)

    # we apply the bigram model to our documents
    return bigram_mod


def create_bigrams(bigram_model, documents: List[List[str]]):
    return [bigram_model[doc] for doc in documents]

In [64]:
bigram_model = learn_bigrams(negative_words_wo_stopwords + positive_words_wo_stopwords)
negative_words_wo_stopwords_bigrams = create_bigrams(bigram_model, negative_words_wo_stopwords)

In [65]:
print(negative_words_wo_stopwords[10])
print(negative_words_wo_stopwords_bigrams[10])

['small', 'beautiful', 'hotel', 'home', 'nights', 'antigua', 'located', 'conveniently', 'close', 'famous', 'yellow', 'arch', 'antigua', 'one', 'block', 'town', 'square', 'shopping', 'banking', 'restaurants', 'tell', 'outside', 'small', 'hotel', 'beautiful', 'courtyard', 'blooming', 'plants', 'greenery', 'pool', 'also', 'covered', 'breakfast', 'area', 'became', 'gathering', 'place', 'morning', 'mostly', 'free', 'coffee', 'cold', 'water', 'fortunately', 'room', 'upstairs', 'breakfast', 'area', 'even', 'though', 'stairs', 'haul', 'suitcases', 'try', 'reserve', 'room', 'rooms', 'balcony', 'watch', 'volcano', 'fuego', 'erupt', 'every', 'night', 'several', 'benches', 'sit', 'along', 'room']
['small', 'beautiful', 'hotel', 'home', 'nights', 'antigua', 'located', 'conveniently', 'close', 'famous', 'yellow', 'arch', 'antigua', 'one_block', 'town_square', 'shopping', 'banking', 'restaurants', 'tell', 'outside', 'small', 'hotel', 'beautiful', 'courtyard', 'blooming', 'plants', 'greenery', 'pool',

In [66]:
def lemmatization(nlp: English, texts: List[List[str]], allowed_postags: List = None) -> List[List[str]]:
    if allowed_postags is None:
        allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']

    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [67]:
negative_words_wo_stopwords_bigrams_pos = lemmatization(nlp, negative_words_wo_stopwords_bigrams)

In [68]:
print(negative_words_wo_stopwords_bigrams[3])
print(negative_words_wo_stopwords_bigrams_pos[3])

['went', 'hotel', 'directly', 'book', 'room', 'receptionist', 'first', 'gave_us', 'really', 'high', 'price', 'lowered', 'asked', 'much', 'costs', 'qetales', 'gave_us', 'comission', 'rate', 'dollars', 'knew', 'randomly', 'goving', 'us', 'non', 'existing', 'rate', 'lowered', 'real', 'rate', 'agreed', 'price', 'includes', 'breakfast', 'following', 'morning', 'get', 'breakfast', 'received', 'two', 'plain', 'pieces', 'toast', 'fruit', 'theother', 'tables', 'received', 'also', 'eggs', 'plantines', 'asked', 'also', 'get', 'eggs', 'abd', 'plantines', 'replied', 'come', 'get', 'breakfast', 'paid', 'good', 'experience', 'hotel', 'much_better', 'places', 'antigua', 'stay']
['go', 'hotel', 'directly', 'book', 'room', 'receptionist', 'first', 'really', 'high', 'price', 'lower', 'ask', 'much', 'cost', 'comission', 'rate', 'dollar', 'know', 'randomly', 'gove', 'existing', 'rate', 'lower', 'real', 'rate', 'agree', 'price', 'include', 'breakfast', 'follow', 'morning', 'get', 'breakfast', 'receive', 'pl

In [69]:
def tokenize(documents: List[str], bigram_model) -> List[List[str]]:

    document_words = sentences_to_words(documents)
    document_words = remove_stopwords(document_words)
    #document_words = create_bigrams(bigram_model, document_words)
    document_words = lemmatization(nlp, document_words)

    return document_words

In [70]:
positive_words = tokenize(positive_reviews, bigram_model)

In [71]:
negative_words = tokenize(negative_reviews, bigram_model)

In [72]:
print(positive_words)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [73]:
with open(r"data/interim/positive_words.pkl", "wb") as output_file:
    pickle.dump(positive_words, output_file)

with open(r"data/interim/negative_words.pkl", "wb") as output_file:
    pickle.dump(negative_words, output_file)
    


In [74]:
with open(r"data/interim/bigram_model.pkl", "wb") as output_file:
    pickle.dump(bigram_model, output_file)